In [5]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
path = "/content/gdrive/My Drive/fh/Train_NT/Train_Vehicle.csv"
vehicle=pd.read_csv(path)

In [108]:
vehicle.shape

(115344, 3)

In [109]:
vehicle.head()

,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust20179,VehicleID,Vehicle8898
1,Cust21384,VehicleModel,Malibu
2,Cust33335,VehicleMake,Toyota
3,Cust27118,VehicleModel,Neon
4,Cust13038,VehicleID,Vehicle30212


In [110]:
vehicle.describe(include='all')

,CustomerID,VehicleAttribute,VehicleAttributeDetails
count,115344,115344,115344
unique,28836,4,28911
top,Cust20179,VehicleID,Saab
freq,4,28836,2415


In [111]:
#both VehicleAttribute and VehicleAttributeDetails are cat
for x in vehicle.drop('CustomerID',axis=1).columns:
  print(vehicle[x].value_counts())

VehicleID       28836
VehicleModel    28836
VehicleMake     28836
VehicleYOM      28836
Name: VehicleAttribute, dtype: int64
Saab            2415
Suburu          2313
Nissan          2300
Dodge           2263
Chevrolet       2174
                ... 
Vehicle10389       1
Vehicle17695       1
Vehicle2852        1
Vehicle24978       1
Vehicle19680       1
Name: VehicleAttributeDetails, Length: 28911, dtype: int64


In [ ]:
#VehicleAttribute has multiple vehicle data:VehicleID,VehicleModel,VehicleMake,VehicleYOM..To split them into separate columns using dict

In [7]:
vehicle['VehicleAttribute']=vehicle['VehicleAttribute'].astype('category')

In [10]:
new_v=pd.DataFrame(vehicle['CustomerID'].unique(),columns=['CustomerID'])

df=vehicle.groupby(by='VehicleAttribute')
#by_id=pd.DataFrame(df.get_group('VehicleID'))
#attr_details=pd.Series(by_id.VehicleAttributeDetails)
#new_v=new_v.merge(by_id,on='CustomerID',how='left')

by_model=pd.DataFrame(df.get_group('VehicleModel'))
model_details=pd.Series(by_model.VehicleAttributeDetails)
new_v=new_v.merge(by_model,on='CustomerID',how='inner')

by_make=pd.DataFrame(df.get_group('VehicleMake'))
make_details=pd.Series(by_make.VehicleAttributeDetails)
new_v=new_v.merge(by_make,on='CustomerID',how='inner')

by_yom=pd.DataFrame(df.get_group('VehicleYOM'))
yom_details=pd.Series(by_yom.VehicleAttributeDetails)
new_v=new_v.merge(by_yom,on='CustomerID',how='inner')

#new_v.columns=['CustomerID','VID','ID','VM','Model','VMK','Make','VYOM','YOM']
new_v.columns=['CustomerID','VM','Model','VMK','Make','VYOM','YOM']

In [11]:
new_v

,CustomerID,VM,Model,VMK,Make,VYOM,YOM
0,Cust20179,VehicleModel,Forrestor,VehicleMake,BMW,VehicleYOM,2011
1,Cust21384,VehicleModel,Malibu,VehicleMake,Chevrolet,VehicleYOM,2010
2,Cust33335,VehicleModel,Highlander,VehicleMake,Toyota,VehicleYOM,2001
3,Cust27118,VehicleModel,Neon,VehicleMake,Saab,VehicleYOM,1998
4,Cust13038,VehicleModel,Maxima,VehicleMake,Toyota,VehicleYOM,2000
...,...,...,...,...,...,...,...
28831,Cust38038,VehicleModel,Legacy,VehicleMake,Suburu,VehicleYOM,2015
28832,Cust19790,VehicleModel,Highlander,VehicleMake,Toyota,VehicleYOM,2004
28833,Cust24765,VehicleModel,Camry,VehicleMake,Toyota,VehicleYOM,2008
28834,Cust10942,VehicleModel,95,VehicleMake,Saab,VehicleYOM,2010


In [32]:
vehicle=pd.DataFrame()
vehicle['CustomerID']=new_v['CustomerID']
#vehicle['VehicleID']=new_v['ID'].astype('category')
vehicle['VehicleModel']=new_v['Model'].astype('category')
vehicle['VehicleMake']=new_v['Make'].astype('category')
vehicle['VehicleYOM']=new_v['YOM'].astype('category')
vehicle.head()

,CustomerID,VehicleModel,VehicleMake,VehicleYOM
0,Cust20179,Forrestor,BMW,2011
1,Cust21384,Malibu,Chevrolet,2010
2,Cust33335,Highlander,Toyota,2001
3,Cust27118,Neon,Saab,1998
4,Cust13038,Maxima,Toyota,2000


In [13]:
vehicle.shape

(28836, 4)

In [121]:
#to check for missing values after converting them to cat
vehicle.dtypes

CustomerID        object
VehicleID       category
VehicleModel    category
VehicleMake     category
VehicleYOM      category
dtype: object

In [14]:
for x in vehicle.drop('CustomerID',axis=1).columns:
  print(vehicle[x].value_counts())
  #there are missing values in VehicleMake:'???'

RAM               1344
Wrangler          1261
A3                1102
MDX               1054
Jetta             1037
Neon               928
Pathfinder         919
Passat             888
Legacy             887
92x                859
Malibu             828
95                 820
A5                 812
F150               797
Forrestor          784
Camry              771
Tahoe              736
93                 724
Maxima             722
Grand Cherokee     718
Escape             706
Ultima             698
E400               695
X5                 691
TL                 684
Silverado          668
Fusion             650
Highlander         633
Civic              604
ML350              599
Impreza            562
CRV                542
Corolla            530
M5                 509
C300               477
X6                 454
3 Series           436
RSX                397
Accord             310
Name: VehicleModel, dtype: int64
Saab          2415
Suburu        2313
Nissan        2300
Dodge        

In [33]:
#to replace missing values in VehicleMake with np.nan
vehicle.VehicleMake.replace('???',np.nan,inplace=True)

In [34]:
#binning years
yom=vehicle.VehicleYOM.to_list()
new_yom=[]
for x in range(len(yom)):
  if(yom[x]<='2000'):
    new_yom.append('0')
  elif('2000'<yom[x]<='2005'):
    new_yom.append('1')
  elif('2005'<yom[x]<='2010'):
    new_yom.append('2')
  else:
    new_yom.append('3')

vehicle.VehicleYOM=new_yom

In [37]:
vehicle.to_csv('vehicle.csv',index=False)